In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j pypdf

In [ ]:
import os
from langchain_core.prompts.prompt import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain

In [ ]:
os.environ["OPENAI_API_KEY"] = " "
os.environ["NEO4J_URI"] = " "
os.environ["NEO4J_USERNAME"] = " "
os.environ["NEO4J_PASSWORD"] = " "

In [ ]:
docs = []
for filename in os.listdir("/content/drive/MyDrive/pdf/"):
    filepath = os.path.join("/content/drive/MyDrive/pdf", filename)
    loader = PyPDFLoader(filepath).load()
    text_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=50)
    documents = text_splitter.split_documents(loader)
    # documents = loader.load_and_split(text_splitter)
    docs.extend(documents)
print(docs)

In [ ]:
db = Neo4jVector.from_documents(
    docs, OpenAIEmbeddings(),
    index_name="health_report",
    url=os.environ.get("NEO4J_URI"),
    username=os.environ.get("NEO4J_USERNAME"),
    password=os.environ.get("NEO4J_PASSWORD"),
    create_id_index=True,
)

In [ ]:
db = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=os.environ.get("NEO4J_URI"),
    username=os.environ.get("NEO4J_USERNAME"),
    password=os.environ.get("NEO4J_PASSWORD"),
    index_name="health_report"
)

In [ ]:
retriever = db.as_retriever()

template = """你是護士，只能回答健康檢查的問題，其他問題一律不回答。回答不出來的就不回答。必須以繁體中文回答問題。
SOURCES:
{question}
{summaries}
"""
GERMAN_QA_PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])
GERMAN_DOC_PROMPT = PromptTemplate(
    template="Inhalt: {page_content}\nQuelle: {source}",
    input_variables=["page_content", "source"])

qa_chain = load_qa_with_sources_chain(ChatOpenAI(temperature=0), chain_type="stuff",
                                      prompt=GERMAN_QA_PROMPT,
                                      document_prompt=GERMAN_DOC_PROMPT)


chain = RetrievalQAWithSourcesChain(combine_documents_chain=qa_chain, retriever=retriever,
                                     reduce_k_below_max_tokens=True, max_tokens_limit=3375,
                                     return_source_documents=True)

In [ ]:
def ask_question_with_context(question, chat_history):
    chat_history=[]
    query = ""
    result = chain({"question": question}, return_only_outputs=True)

    print("answer:", result["answer"])
    chat_history = [(query, result["answer"])]
    return chat_history

In [ ]:
chat_history = []
while True:
    query = input('you: ')
    if query == 'q':
        break
    chat_history = ask_question_with_context(query, chat_history)


answer: 對於預防嬰兒得遺傳高血壓，可以戒菸（含電子煙）、戒酒，並且適時地用健康的方式紓解壓力，如控制體重，避免肥胖，限制鈉鹽、菸酒、咖啡因，保持生活作息正常。根據醫師指示，按時服用降血壓藥物也是重要的控制方式。高血壓的控制對於遺傳高血壓的預防有一定的幫助。
answer: 根據資料顯示，有遺傳高血壓的女性42歲，生出有高血壓的嬰兒的風險會增加。遺傳因素是影響高血壓風險的重要因素之一。因此，建議在懷孕前向醫師諮詢，以確保適當的護理和監控。
answer: 患有三高的人應該遵循低脂飲食原則，避免食用含糖高的食物，飲食控制加運動，適當選擇運動方式，按時注射胰島素或服用口服降血糖藥，並記錄血糖、血壓、血脂等指標，定期追蹤身體狀況。另外，避免食用含糖高的食物，多選用富含纖維質的食物，避免飲酒，並適當調整生活型態。
answer: 對於三高（高血壓、高血糖、高膽固醇）如果不加以控制，可能會導致各種併發症，包括全身大、小血管及神經系統的併發症。因此，必須持續地控制三高，包括定時服藥、控制體重、避免肥胖、限制鈉鹽、菸酒、咖啡因等，以維持健康。
answer: 健康檢查的項目並不是越多越好，而是要根據個人的健康狀況和需求來選擇適合的檢查項目。費用高低也不一定代表檢查的品質，重要的是選擇對自己有意義和價值的檢查項目。對於第一次健康檢查者，建議可以從基本的檢查項目開始，例如血壓、血糖、膽固醇等，再根據醫師的建議進行進一步的檢查。希望能幫助您做出明智的健康檢查選擇。
answer: 腦中風和高血脂之間有關聯。高血脂可能導致腦中風、心肌梗塞等嚴重併發症。具有高血脂症狀的人可能出現腦中風、腎功能減退、心絞痛、心肌梗塞、動脈瘤或四肢末梢壞死等症狀。然而，有這些症狀不一定代表患有高血脂症，建議如有相關症狀應尋求心臟內科醫師的診治。具有中風家族史的人，通過運動、飲食控制和體重管理，有機會降低高血壓和相關併發症的風險。代謝症候群的人有較高風險患上中風和心肌梗塞，其中腹部肥胖是其中一個指標。
answer: 根據您提供的數據，總膽固醇為200mg/dl，三酸甘油酯為110mg/dl，根據標準，總膽固醇≥200 mg/dL，三酸甘油脂≥200 mg/dL被認為是異常的。因此，根據您提供的數據，確實可以被歸類為高血脂症。高血脂症可能導致粥狀動脈硬化等健康問題，建議您咨詢醫生進一步評估和管理。
ans